In [1]:
import os
import sys
import pyspark
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, filter
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import ClusteringEvaluator
import requests
from pyspark.sql.types import DoubleType

/home/sathvik/.local/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
spark = SparkSession.builder.appName("Anomaly detection using KMeans").getOrCreate()

24/11/03 23:32:11 WARN Utils: Your hostname, sathvik-HP-EliteBook-x360-1030-G2 resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface wlp58s0)
24/11/03 23:32:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/03 23:32:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/03 23:32:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
local_path = "../Data/iris.csv"

response = requests.get(url)
with open(local_path, 'wb') as f:
  f.write(response.content)

In [4]:
df = spark.read.csv(local_path, header=False, inferSchema=True)
columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]

df = df.toDF(*columns)
df.show(5)

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|      class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
+------------+-----------+------------+-----------+-----------+
only showing top 5 rows



In [5]:
df.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- class: string (nullable = true)



In [6]:
df.describe().show()

24/11/03 23:33:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-------------------+------------------+------------------+--------------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|         class|
+-------+------------------+-------------------+------------------+------------------+--------------+
|  count|               150|                150|               150|               150|           150|
|   mean| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|          NULL|
| stddev|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|          NULL|
|    min|               4.3|                2.0|               1.0|               0.1|   Iris-setosa|
|    max|               7.9|                4.4|               6.9|               2.5|Iris-virginica|
+-------+------------------+-------------------+------------------+------------------+--------------+



In [7]:
df = df.dropna()

In [8]:
assembler = VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"],
                            outputCol="features")

assembled_df = assembler.transform(df)

scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(assembled_df)
scaled_df = scaler_model.transform(assembled_df)

In [9]:
kmeans = KMeans(k=3, seed=1, featuresCol="scaled_features", predictionCol="clusters")
model = kmeans.fit(scaled_df)
clustered_df = model.transform(scaled_df)

clustered_df.show(5)

+------------+-----------+------------+-----------+-----------+-----------------+--------------------+--------+
|sepal_length|sepal_width|petal_length|petal_width|      class|         features|     scaled_features|clusters|
+------------+-----------+------------+-----------+-----------+-----------------+--------------------+--------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|[6.15892840883878...|       1|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|[5.9174018045706,...|       1|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|[5.67587520030241...|       1|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|[5.55511189816831...|       1|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|[6.03816510670469...|       1|
+------------+-----------+------------+-----------+-----------+-----------------+--------------------+--

In [10]:

centers = model.clusterCenters()
for i , center in enumerate(centers):
  print(f"Cluster {i+1} Centers: {center}")

Cluster 1 Centers: [8.08674985 7.05728816 3.07079977 2.53967705]
Cluster 2 Centers: [6.05788156 7.93556612 0.83163112 0.31822541]
Cluster 3 Centers: [6.8887588  6.07660788 2.38900964 1.7461704 ]


In [16]:
def euclidean_distance(features, center):
    return float(np.sqrt(np.sum((np.array(features) - np.array(center)) ** 2)))

In [17]:
distance_udf = udf(lambda point, cluster: euclidean_distance(point, centers[cluster]), DoubleType())

distance_df = clustered_df.withColumn("distance", distance_udf(col("scaled_features"), col("clusters")))

threshold = 1.0
anomalies = distance_df.filter(col("distance") > threshold)
anomalies.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- class: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)
 |-- clusters: integer (nullable = false)
 |-- distance: double (nullable = true)



In [18]:
distance_df = distance_df.withColumn("is_anomaly", col("distance") > threshold)
num_anomalies = distance_df.filter(col("is_anomaly")).count()
num_normal = distance_df.filter(~col("is_anomaly")).count()
total_count = distance_df.count()

print(f"Total number of anomalies: {num_anomalies}")
print(f"Number of normal data points: {num_normal}")
print(f"Total data points: {total_count}")

/home/sathvik/.local/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Total number of anomalies: 46
Number of normal data points: 104
Total data points: 150


In [19]:
os.remove(local_path)
spark.stop()